In [300]:
### Extra code to reuse

## Exporting data to CSV
# combined_data.to_csv('file_name.csv')

## Finding what the Data Type a column is.
# data_type = IMDb_Basics['isAdult'].dtype
# print(data_type)

## Filtering dataframes for a specific values
# filtered_df = IMDB_Basics[IMDB_Basics['primaryTitle'].str.contains('Bond', case=False, na=False)]
# print(filtered_df)

# Optionally save the combined dataframe to a new CSV file
# output_file = path_to_files + 'combined_data.csv'
# combined_df.to_csv(output_file, index=False)
# print(f"Combined CSV saved to {output_file}")

# Importing all Datasets

In [301]:
import os
import glob
import math
import numpy as np
import pandas as pd
from datetime import datetime

os.chdir('/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/ECON 5813 - Economic Analytics I/Project Proposal/Data')

# 1: Stocks
Amazon_Stock = pd.read_csv('Amazon Stock.csv')
Apple_Stock = pd.read_csv('Apple Stock.csv')
ATT_Stock = pd.read_csv('AT&T Stock.csv')
Comcast_Stock = pd.read_csv('Comcast Stock.csv')
Lions_Gate_Entertainment_Historical = pd.read_csv('Lions Gate Entertainment Historical.csv')
Lions_Gate_Historical = pd.read_csv('Lions Gate Historical.csv')
Netflix_Stock_Data = pd.read_csv('Netflix Stock Data.csv')
Paramount_Global_Historical_Prices = pd.read_csv('Paramount Global Historical Prices.csv')
Sony_Stock = pd.read_csv('Sony Stock.csv')
The_Walt_Disney_Company_Historical_Prices = pd.read_csv('The Walt Disney Company Historical Prices.csv')
Warner_Bros_Discovery = pd.read_csv('Warner Bros Discovery.csv')
Vanguard_500_Historical = pd.read_csv('Vanguard 500 Historical.csv')

#############

# 2: Economic Indicators 
GDP = pd.read_csv('GDP.csv') # Billions of Dollars & Seasonally Adjusted Annual Rate
Unemployment_Rate = pd.read_csv('Unemployment Rate US.csv') # This is Percent & Seasonally Adjusted

#############

# 3. IMDB Data -- More about the dataset below in markdown
IMDb_Basics = pd.read_csv('IMDb Basics.tsv', sep='\t')
IMDb_Name_Basics = pd.read_csv('IMDb Name Basics.tsv', sep='\t')
IMDb_Ratings = pd.read_csv('IMDb Ratings.tsv', sep='\t')
IMDb_Title_Akas = pd.read_csv('IMDb Title Akas.tsv', sep='\t')
IMDb_Title_Crew = pd.read_csv('IMDb Title Crew.tsv', sep='\t')
Title_Episode = pd.read_csv('Title Episode.tsv', sep='\t')
Title_Principals = pd.read_csv('Title Principals.tsv', sep='\t')

#############

# 4: Gross Movie revenue data
# Function to try multiple date formats
def parse_dates(date):
    for fmt in ('%d-%b', '%Y-%m-%d', '%m/%d/%Y', '%d/%m/%Y'):  # Add or modify formats as needed
        try:
            return pd.to_datetime(date, format=fmt)
        except ValueError:
            continue
    # Return the original date if no format matched
    return pd.NaT

# Path to the directory where all your CSV files are stored
path_to_files = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/ECON 5813 - Economic Analytics I/Project Proposal/Data/Gross Data/'

# Use glob to match the pattern for all CSV files. The * wildcard will match any characters
file_pattern = path_to_files + '/* *.csv'
file_list = glob.glob(file_pattern)

# Create an empty list to hold your dataframes
dataframes_list = []

# Loop over the list of file paths & names
for file_name in file_list:
    # Read the current file into a dataframe
    df = pd.read_csv(file_name)
    
    # Extract month and year from the filename to create a new 'Date' column if needed
    # This can help if the 'Release Date' column isn't consistent
    month_year = file_name.split('/')[-1].replace('.csv', '')
    try:
        # Trying to convert 'month year' to a datetime object, assuming format 'Month YYYY'
        df['Date'] = datetime.strptime(month_year, '%B %Y')
    except ValueError:
        print(f"Skipping file {file_name}: Date format issue")
        continue
    
    # Append the dataframe to the list
    dataframes_list.append(df)

# Combine all dataframes into one
Movie_Revenues = pd.concat(dataframes_list, ignore_index=True)

# Apply the custom date parser to the 'Release Date' column
Movie_Revenues['Release Date'] = Movie_Revenues['Release Date'].apply(parse_dates)

# Now, sort by 'Release Date' if possible, otherwise, by the new 'Date' column
if Movie_Revenues['Release Date'].isnull().all():
    Movie_Revenues = Movie_Revenues.sort_values('Date')
else:
    Movie_Revenues = Movie_Revenues.sort_values('Release Date')

/var/folders/7_/z9jxc0jj6fx4snlcjg1wby4r0000gn/T/ipykernel_2075/3883758322.py:33: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  IMDb_Basics = pd.read_csv('IMDb Basics.tsv', sep='\t')
/var/folders/7_/z9jxc0jj6fx4snlcjg1wby4r0000gn/T/ipykernel_2075/3883758322.py:36: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  IMDb_Title_Akas = pd.read_csv('IMDb Title Akas.tsv', sep='\t')


---------------

# Modeling Stocks Data

In [302]:
# Adding company to each individual stick dataframe to the respective company name.
Amazon_Stock['Company'] = 'Amazon'
Apple_Stock['Company'] = 'Apple'
ATT_Stock['Company'] = 'AT&T'
Comcast_Stock['Company'] = 'Comcast'
Lions_Gate_Entertainment_Historical['Company'] = 'Lions Gate Entertainment' #look into the 2 lionsgate datasets I have and what problems it could create.
Lions_Gate_Historical['Company'] = 'Lions Gate'
Netflix_Stock_Data['Company'] = 'Netflix'
Paramount_Global_Historical_Prices['Company'] = 'Paramount Global'
Sony_Stock['Company'] = 'Sony'
The_Walt_Disney_Company_Historical_Prices['Company'] = 'Walt Disney'
Warner_Bros_Discovery['Company'] = 'Warner Bros Discovery'
Vanguard_500_Historical['Company'] = 'Vanguard_500_Historical'

# Convert 'Date' columns to datetime for all dataframes before merging
# Note: If your date formats are inconsistent, you may need to handle them individually.
company_stocks = [Amazon_Stock, Apple_Stock, ATT_Stock, Comcast_Stock, Lions_Gate_Entertainment_Historical, Lions_Gate_Historical, Netflix_Stock_Data, Paramount_Global_Historical_Prices, Sony_Stock, The_Walt_Disney_Company_Historical_Prices, Warner_Bros_Discovery, Vanguard_500_Historical]

for stock_df in company_stocks:
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])

# Define the list of company names
company_names = ['Amazon', 'Apple', 'AT&T', 'Comcast', 'Lions Gate Entertainment', 'Lions Gate', 'Netflix', 'Paramount Global', 'Sony', 'Walt Disney', 'Warner Bros Discovery', 'Vanguard 500']

# Initialize the combined DataFrame
all_stocks = pd.DataFrame()

# Loop through each company DataFrame
for stock_df, company_name in zip(company_stocks, company_names):
    # Rename columns to include the company name (excluding 'Date')
    cols_to_rename = {col: f"{col}_{company_name}" for col in stock_df.columns if col != 'Date'}
    df_renamed = stock_df.rename(columns=cols_to_rename)
    
    # Merge with the combined DataFrame
    if all_stocks.empty:
        all_stocks = df_renamed
    else:
        all_stocks = all_stocks.merge(df_renamed, on='Date', how='outer')

# all_stocks now contains all the data with renamed columns and one 'Date' column
all_stocks.sort_values('Date', inplace=True)

# Check the result
print(all_stocks.head())


            Date  Open_Amazon  High_Amazon  Low_Amazon  Close_Amazon  \
12775 1962-01-02          NaN          NaN         NaN           NaN   
12776 1962-01-03          NaN          NaN         NaN           NaN   
12777 1962-01-04          NaN          NaN         NaN           NaN   
12778 1962-01-05          NaN          NaN         NaN           NaN   
12779 1962-01-08          NaN          NaN         NaN           NaN   

       Adj Close_Amazon  Volume_Amazon Company_Amazon  Open_Apple  High_Apple  \
12775               NaN            NaN            NaN         NaN         NaN   
12776               NaN            NaN            NaN         NaN         NaN   
12777               NaN            NaN            NaN         NaN         NaN   
12778               NaN            NaN            NaN         NaN         NaN   
12779               NaN            NaN            NaN         NaN         NaN   

       ...  Adj Close_Warner Bros Discovery  Volume_Warner Bros Discovery  \
127

/var/folders/7_/z9jxc0jj6fx4snlcjg1wby4r0000gn/T/ipykernel_2075/945722630.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  stock_df['Date'] = pd.to_datetime(stock_df['Date'])


In [303]:
def parse_dates(date):
    for fmt in ("%Y-%m-%d", "%m/%d/%y"):  # Add or adjust formats as needed
        try:
            return pd.to_datetime(date, format=fmt)
        except ValueError:
            continue
    return pd.NaT  # return Not a Time for unparseable formats

all_stocks['Date'] = all_stocks['Date'].apply(parse_dates)

## all_stocks table and it's statistics

In [304]:
# Code to check date columns

# 1. Check for NaT values:
# If you used errors='coerce' during the conversion, you can check for NaT values, which would indicate conversion failures.
# nat_counts = all_stocks['Date'].isna().sum()
# print(f"Number of NaT values in 'Date': {nat_counts}")

# 2. Check the range of dates:
# Look at the minimum and maximum values to ensure they are within expected bounds.
# print("Earliest date:", all_stocks['Date'].min())
# print("Latest date:", all_stocks['Date'].max())

# 3. Sample random dates:
# Random sampling of dates can give you a quick spot check to see if dates look correctly formatted.
# print(all_stocks['Date'].sample(10))

# 4. Validate with known dates:
# If you know certain dates should exist in the dataset, check those specifically.
# known_dates = ["1997-05-15", "2005-07-08"]  # Example dates you expect to find
# for date in known_dates:
#     if pd.to_datetime(date) not in all_stocks['Date'].values:
#         print(f"Date {date} is not found or incorrectly formatted.")
#     else:
#         print(f"Date {date} is correctly formatted.")

# 5. Check for duplicates:
# Ensuring that there are no duplicate dates which could suggest an issue with the conversion.
# duplicate_dates = all_stocks['Date'].duplicated().sum()
# print(f"Number of duplicate dates: {duplicate_dates}")


In [305]:
all_stocks.head()

,Date,Open_Amazon,High_Amazon,Low_Amazon,Close_Amazon,Adj Close_Amazon,Volume_Amazon,Company_Amazon,Open_Apple,High_Apple,...,Adj Close_Warner Bros Discovery,Volume_Warner Bros Discovery,Company_Warner Bros Discovery,Open_Vanguard 500,High_Vanguard 500,Low_Vanguard 500,Close_Vanguard 500,Adj Close_Vanguard 500,Volume_Vanguard 500,Company_Vanguard 500
12775,1962-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12776,1962-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12777,1962-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12778,1962-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12779,1962-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [306]:
all_stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15554 entries, 12775 to 10988
Data columns (total 85 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Date                                15554 non-null  datetime64[ns]
 1   Open_Amazon                         6648 non-null   float64       
 2   High_Amazon                         6648 non-null   float64       
 3   Low_Amazon                          6648 non-null   float64       
 4   Close_Amazon                        6648 non-null   float64       
 5   Adj Close_Amazon                    6648 non-null   float64       
 6   Volume_Amazon                       6648 non-null   float64       
 7   Company_Amazon                      6648 non-null   object        
 8   Open_Apple                          10800 non-null  float64       
 9   High_Apple                          10800 non-null  float64       
 10  Low_Apple              

In [307]:
all_stocks.dtypes[6]

/var/folders/7_/z9jxc0jj6fx4snlcjg1wby4r0000gn/T/ipykernel_2075/979003749.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  all_stocks.dtypes[6]


dtype('float64')

In [308]:
all_stocks.isnull().sum()

Date                          0
Open_Amazon                8906
High_Amazon                8906
Low_Amazon                 8906
Close_Amazon               8906
                          ...  
Low_Vanguard 500          12257
Close_Vanguard 500        12257
Adj Close_Vanguard 500    12257
Volume_Vanguard 500       12257
Company_Vanguard 500      12257
Length: 85, dtype: int64

In [309]:
all_stocks.isnull().mean()

Date                      0.000000
Open_Amazon               0.572586
High_Amazon               0.572586
Low_Amazon                0.572586
Close_Amazon              0.572586
                            ...   
Low_Vanguard 500          0.788029
Close_Vanguard 500        0.788029
Adj Close_Vanguard 500    0.788029
Volume_Vanguard 500       0.788029
Company_Vanguard 500      0.788029
Length: 85, dtype: float64

In [310]:
all_stocks.nunique()

Date                      15554
Open_Amazon                5919
High_Amazon                5875
Low_Amazon                 5879
Close_Amazon               6017
                          ...  
Low_Vanguard 500           2983
Close_Vanguard 500         3020
Adj Close_Vanguard 500     3213
Volume_Vanguard 500        3212
Company_Vanguard 500          1
Length: 85, dtype: int64

In [311]:
all_stocks.value_counts()

Date        Open_Amazon  High_Amazon  Low_Amazon  Close_Amazon  Adj Close_Amazon  Volume_Amazon  Company_Amazon  Open_Apple  High_Apple  Low_Apple   Close_Apple  Adj Close_Apple  Volume_Apple  Company_Apple  Open_AT&T  High_AT&T  Low_AT&T  Close_AT&T  Adj Close_AT&T  Volume_AT&T  Company_AT&T  Open_Comcast  High_Comcast  Low_Comcast  Close_Comcast  Adj Close_Comcast  Volume_Comcast  Company_Comcast  Open_Lions Gate Entertainment  High_Lions Gate Entertainment  Low_Lions Gate Entertainment  Close_Lions Gate Entertainment  Adj Close_Lions Gate Entertainment  Volume_Lions Gate Entertainment  Company_Lions Gate Entertainment  Open_Lions Gate  High_Lions Gate  Low_Lions Gate  Close_Lions Gate  Adj Close_Lions Gate  Volume_Lions Gate  Company_Lions Gate  Open_Netflix  High_Netflix  Low_Netflix  Close_Netflix  Adj Close_Netflix  Volume_Netflix  Company_Netflix  Open_Paramount Global  High_Paramount Global  Low_Paramount Global  Close_Paramount Global  Adj Close_Paramount Global  Volume_Paramo

In [312]:
all_stocks.memory_usage(deep=True)

Index                     124432
Date                      124432
Open_Amazon               124432
High_Amazon               124432
Low_Amazon                124432
                           ...  
Low_Vanguard 500          124432
Close_Vanguard 500        124432
Adj Close_Vanguard 500    124432
Volume_Vanguard 500       124432
Company_Vanguard 500      629608
Length: 86, dtype: int64

In [313]:
#Descriptive Statistics for All Stocks.
print(all_stocks.describe())

                                Date  Open_Amazon  High_Amazon   Low_Amazon  \
count                          15554  6648.000000  6648.000000  6648.000000   
mean   1992-12-01 11:34:37.973511680    33.465443    33.868688    33.026142   
min              1962-01-02 00:00:00     0.070313     0.072396     0.065625   
25%              1977-07-12 06:00:00     2.022719     2.063875     1.989750   
50%              1992-11-26 00:00:00     6.857250     6.960000     6.766000   
75%              2008-05-05 18:00:00    42.573249    42.694751    42.343500   
max              2023-10-16 00:00:00   187.199997   188.654007   184.839493   
std                              NaN    49.394337    49.975037    48.752435   

       Close_Amazon  Adj Close_Amazon  Volume_Amazon    Open_Apple  \
count   6648.000000       6648.000000   6.648000e+03  10800.000000   
mean      33.454436         33.454436   1.408481e+08     19.244714   
min        0.069792          0.069792   9.744000e+06      0.049665   
25%     

---------------------

# Modeling Economic Indicators

In [314]:
#Renaming a column in the Unemployment table
Unemployment_Rate.rename(columns={'LRUNTTTTUSQ156S': 'Unemployment_rate'}, inplace=True)
# Unemployment_Rate.head()

# Convert the dictionaries to DataFrames
GDP = pd.DataFrame(GDP)
Unemployment = pd.DataFrame(Unemployment_Rate)

# Merge the DataFrames on the 'DATE' column
Economic_Indicators = GDP.merge(Unemployment, on='DATE', how='outer')

Economic_Indicators.tail()
# Economic_Indicators.to_csv('file_name.csv')

,DATE,GDP,Unemployment_rate
301,2022-04-01,25544.273,3.600000
302,2022-07-01,25994.639,3.566667
303,2022-10-01,26408.405,3.600000
304,2023-01-01,26813.601,3.500000
305,2023-04-01,27063.012,3.566667


In [315]:
#Descriptive Statistics for All Stocks.
print(Economic_Indicators.describe())

                GDP  Unemployment_rate
count    306.000000         254.000000
mean    7024.590363           5.934121
std     7225.574141           1.682763
min      243.164000           3.400000
25%      798.045750           4.766667
50%     4262.527500           5.650000
75%    12065.473000           6.991667
max    27063.012000          12.966667


# Cleaning all IMDB

<a name="_page0_x69.69_y423.22"></a>**IMDb Dataset Legend**

<Details>

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name. The available datasets are as follows:

<a name="_page0_x60.71_y555.04"></a>**title.akas.tsv.gz**

- titleId (string) - a tconst, an alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- title (string) – the localized title
- region (string) - the region for this version of the title
- language (string) - the language of the title
- types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
- attributes (array) - Additional terms to describe this alternative title, not enumerated
- isOriginalTitle (boolean) – 0: not original title; 1: original title <a name="_page1_x60.71_y183.54"></a>**title.basics.tsv.gz**
- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year ![](Aspose.Words.6ef9d709-0a24-4a63-90ff-a7a4701c44f5.002.png) endYear (YYYY) – TV Series end year. ‘\N’ for all other title types ![](Aspose.Words.6ef9d709-0a24-4a63-90ff-a7a4701c44f5.003.png) runtimeMinutes – primary runtime of the title, in minutes ![](Aspose.Words.6ef9d709-0a24-4a63-90ff-a7a4701c44f5.004.png) genres (string array) – includes up to three genres associated with the title

<a name="_page1_x60.71_y549.05"></a>**title.crew.tsv.gz**

- tconst (string) - alphanumeric unique identifier of the title
- directors (array of nconsts) - director(s) of the given title
- writers (array of nconsts) – writer(s) of the given title

<a name="_page1_x60.71_y680.87"></a>**title.episode.tsv.gz**

- tconst (string) - alphanumeric identifier of episode
- parentTconst (string) - alphanumeric identifier of the parent TV Series
- seasonNumber (integer) – season number the episode belongs to
- episodeNumber (integer) – episode number of the tconst in the TV series

<a name="_page2_x60.71_y99.65"></a>**title.principals.tsv.gz**

- tconst (string) - alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- nconst (string) - alphanumeric unique identifier of the name/person
- category (string) - the category of job that person was in
- job (string) - the specific job title if applicable, else '\N'
- characters (string) - the name of the character played if applicable, else '\N'

<a name="_page2_x60.71_y321.35"></a>**title.ratings.tsv.gz**

- tconst (string) - alphanumeric unique identifier of the title
- averageRating – weighted average of all the individual user ratings
- numVotes - number of votes the title has received

<a name="_page2_x60.71_y453.18"></a>**name.basics.tsv.gz**

- nconst (string) - alphanumeric unique identifier of the name/person
- primaryName (string)– name by which the person is most often credited
- birthYear – in YYYY format
- deathYear – in YYYY format if applicable, else '\N'
- primaryProfession (array of strings)– the top-3 professions of the person
- knownForTitles (array of tconsts) – titles the person is known for



In [316]:
# Changes to the IMBb_Basics and IMDb_Title_Akas tables columns to strings.

# Convert the problematic columns to string data types
column_name_4_IMDb_Basics = IMDb_Basics.columns[4]  # Replace with actual column name
column_name_7_IMDb_Title_Akas = IMDb_Title_Akas.columns[7]  # Replace with actual column name

IMDb_Basics[column_name_4_IMDb_Basics] = IMDb_Basics[column_name_4_IMDb_Basics].astype(str)
IMDb_Title_Akas[column_name_7_IMDb_Title_Akas] = IMDb_Title_Akas[column_name_7_IMDb_Title_Akas].astype(str)


### 1. Cleaning IMDB_Basics
-----------

In [317]:
# Converting the data to different data types

IMDb_Basics['tconst'] = IMDb_Basics['tconst'].astype(str)
IMDb_Basics['primaryTitle'] = IMDb_Basics['primaryTitle'].astype(str)
IMDb_Basics['originalTitle'] = IMDb_Basics['originalTitle'].astype(str)
IMDb_Basics['isAdult'] = IMDb_Basics['isAdult'].astype(bool)

# Handle 'startYear'
IMDb_Basics['startYear'] = IMDb_Basics['startYear'].replace('\\N', np.nan)
IMDb_Basics['startYear'] = IMDb_Basics['startYear'].astype('float').astype('Int64')

# Handle 'endYear'
IMDb_Basics['endYear'] = IMDb_Basics['endYear'].replace('\\N', np.nan)
IMDb_Basics['endYear'] = IMDb_Basics['endYear'].astype('float').astype('Int64')

IMDb_Basics['runtimeMinutes'] = IMDb_Basics['runtimeMinutes'].astype(str)
IMDb_Basics['genres'] = IMDb_Basics['genres'].astype(str)

In [318]:
print(IMDb_Basics.dtypes)

tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult             bool
startYear          Int64
endYear            Int64
runtimeMinutes    object
genres            object
dtype: object


In [319]:
IMDb_Basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,True,1894,<NA>,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,True,1892,<NA>,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,True,1892,<NA>,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,True,1892,<NA>,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,True,1893,<NA>,1,"Comedy,Short"


### 2. Cleaning IMDB_name_Basic
-----------

In [320]:
# Inspecting how the df looks.
IMDb_Name_Basics.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0031983,tt0053137,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0056404,tt0049189,tt0054452,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0080455,tt0077975,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0050976,tt0050986,tt0069467"


In [321]:
# Inspecting the data types
print(IMDb_Name_Basics.dtypes)

nconst               object
primaryName          object
birthYear            object
deathYear            object
primaryProfession    object
knownForTitles       object
dtype: object


In [322]:
# Convert string columns
IMDb_Name_Basics['nconst'] = IMDb_Name_Basics['nconst'].astype(str)
IMDb_Name_Basics['primaryName'] = IMDb_Name_Basics['primaryName'].astype(str)
IMDb_Name_Basics['primaryProfession'] = IMDb_Name_Basics['primaryProfession'].astype(str)
IMDb_Name_Basics['knownForTitles'] = IMDb_Name_Basics['knownForTitles'].astype(str)

# Convert birthYear and deathYear to integer (with NA support)
# First, replace '\N' with NaN
IMDb_Name_Basics['birthYear'].replace('\\N', pd.NA, inplace=True)
IMDb_Name_Basics['deathYear'].replace('\\N', pd.NA, inplace=True)

# Convert to Int64 (capital "I" to support NaN)
IMDb_Name_Basics['birthYear'] = IMDb_Name_Basics['birthYear'].astype('Int64')
IMDb_Name_Basics['deathYear'] = IMDb_Name_Basics['deathYear'].astype('Int64')


In [323]:
# Finally looking at the changed data types
print(IMDb_Name_Basics.dtypes)

nconst               object
primaryName          object
birthYear             Int64
deathYear             Int64
primaryProfession    object
knownForTitles       object
dtype: object


### 3. Cleaning IMDb_Ratings DF
------------

In [324]:
IMDb_Ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2002
1,tt0000002,5.8,269
2,tt0000003,6.5,1892
3,tt0000004,5.5,178
4,tt0000005,6.2,2679


In [325]:
# Inspecting the data types
IMDb_Ratings.dtypes

tconst            object
averageRating    float64
numVotes           int64
dtype: object

### 4. Cleaning IMDb_Title_Akas
-----------

In [326]:
IMDb_Title_Akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [327]:
IMDb_Title_Akas.dtypes

titleId            object
ordering            int64
title              object
region             object
language           object
types              object
attributes         object
isOriginalTitle    object
dtype: object

### 5. Cleaning IMDb_Title_Crew
-----------

In [328]:
IMDb_Title_Crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [329]:
IMDb_Title_Crew.dtypes

tconst       object
directors    object
writers      object
dtype: object

### 6. Cleaning IMDb_Episode
-----------

In [330]:
Title_Episode.head(20)

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16
5,tt0043693,tt0989125,2,8
6,tt0043710,tt0989125,3,3
7,tt0044093,tt0959862,1,6
8,tt0044668,tt0044243,2,16
9,tt0044901,tt0989125,3,46


### 7. Cleaning IMDb_Principals
-----------

In [331]:
Title_Principals.head(20)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
5,tt0000003,1,nm0721526,director,\N,\N
6,tt0000003,2,nm1770680,producer,producer,\N
7,tt0000003,3,nm1335271,composer,\N,\N
8,tt0000003,4,nm5442200,editor,\N,\N
9,tt0000004,1,nm0721526,director,\N,\N


# Testing

In [332]:
IMDb_Basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,True,1894,<NA>,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,True,1892,<NA>,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,True,1892,<NA>,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,True,1892,<NA>,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,True,1893,<NA>,1,"Comedy,Short"


In [333]:
IMDb_Ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2002
1,tt0000002,5.8,269
2,tt0000003,6.5,1892
3,tt0000004,5.5,178
4,tt0000005,6.2,2679


In [334]:
os.chdir('/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/ECON 5813 - Economic Analytics I/Project Proposal/Data/Gross Data')
January_2023 = pd.read_csv('January 2023.csv')
January_2023.head(10)

,Rank,Release,Gross,Theaters,Total Gross,Release Date,Distributor
0,1.0,Avatar: The Way of Water,"$222,528,552","4,340","$684,075,767",16-Dec,20th Century Studios
1,2.0,Puss in Boots: The Last Wish,"$86,503,985","4,121","$185,535,345",21-Dec,Universal Pictures
2,3.0,M3GAN,"$83,058,490","3,628","$95,043,350",6-Jan,Universal Pictures
3,4.0,A Man Called Otto,"$47,561,111","3,957","$64,267,657",30-Dec,Columbia Pictures
4,5.0,Plane,"$26,138,048","3,060","$32,111,181",13-Jan,Lions Gate Films
5,6.0,Missing,"$18,575,107","3,025","$32,502,025",20-Jan,Sony Pictures Entertainment (SPE)
6,7.0,Black Panther: Wakanda Forever,"$16,656,634","4,396","$453,829,060",11-Nov,Walt Disney Studios Motion Pictures
7,8.0,Pathaan,"$10,822,853",697,"$17,487,476",25-Jan,Yash Raj Films USA Inc.
8,9.0,Whitney Houston: I Wanna Dance with Somebody,"$10,224,940","3,625","$23,708,080",23-Dec,TriStar Pictures
9,10.0,The Whale,"$9,807,231","1,721","$17,463,630",9-Dec,A24


# 4. Cleaning Movies_Revenues dataset

In [335]:
# The Gross Revenue Dataset
print(Movie_Revenues.head())

        Rank             Release      Gross Theaters Total Gross Release Date  \
7618   115.0  Chimes at Midnight   $18,111         4   $126,724    1900-01-01   
7001   115.0            Yosemite   $11,090         4    $13,931    1900-01-01   
6941    59.0      Ho Mann Jahaan  $201,591        23   $208,611    1900-01-01   
12413   74.0      Only Yesterday   $70,884        44   $453,243    1900-01-01   
6197   122.0      Only Yesterday   $19,125        44   $453,243    1900-01-01   

          Distributor       Date  
7618      Janus Films 2016-03-01  
7001   Monterey Media 2016-01-01  
6941   Eros Worldwide 2016-01-01  
12413           GKIDS 2016-02-01  
6197            GKIDS 2016-04-01  
